In [1]:
import os
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy.polynomial import Polynomial
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from tqdm import tqdm
import logging
import seaborn as sns
import matplotlib.pyplot as plt
from tensorboard.plugins.hparams import api as hp
from utilities import get_npz_data, get_bci_iii_data, get_project_data

tf.random.set_seed(333)
logger = logging.getLogger('1d-AX-LSTM')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

%load_ext tensorboard

# TensorBoard
------
#### Used to track the gridsearch and visualize the process of the system.

In [2]:
%tensorboard --logdir LOG

Launching TensorBoard...

KeyboardInterrupt: 

In [3]:
LOG_PATH = f'{os.getcwd()}/LOG/1dAX_LSTM'

HYBRID_DATA_PATH = f'{os.getcwd()}/Data/Hybrid'
BCI_IV_2a_DATA_PATH = f'{os.getcwd()}/Data/BCI'
BCI_III_IVa_DATA_PATH = f'{os.getcwd()}/Data/BCI III IVa'

EEG, TARGET = get_project_data(HYBRID_DATA_PATH, [2.0, 3.0], sec=3, offset=1)

if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

49 files found.

Fetching from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/Hybrid/Alex-1Cq5YjzM3eu-XaCck7Z5jQ_EPOCPLUS_37553_2022.02.24T14.45.30+01.00.csv
Duration 25.64453125s

Fetching from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/Hybrid/Alex-5Ve2WqgPNmhLrb3_6rBndA_EPOCPLUS_37553_2022.02.24T14.58.44+01.00.csv
Duration 25.390625s

Fetching from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/Hybrid/Alex-b-e3bztqNa3Ce_biLVPMMw_EPOCPLUS_37553_2022.02.24T15.06.54+01.00.csv
Duration 25.9140625s

Fetching from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/Hybrid/Alex-bb8b0OAsspxDk2jwFWPFOQ_EPOCPLUS_37553_2022.02.24T14.19.40+01.00.csv
Duration 25.609375s

Fetching from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/Hybrid/Alex-gXqLz0RyIYznyJCHuEiFxg_EPOCPLUS_37553_2022.02.24T14.05.13+01.00.csv
Duration 26.05078125s

Fetching from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/Hybrid/Alex-tHSv5BsrzPVomLbFf23Lww_EPOCPLUS_37553_2022.02.24T15.08.56+01.00.csv
Duration 26.16015625s

Fetching from /upb/scratch/users/d

# Preprocessing
----
#### The first preprocessing step is the normalisation. The scheme for this is the follwing. Let us consider $\mathbf{X}_{\mathrm{EEG}}=[\mathbf{x}^{(1)},\ldots, \mathbf{x}^{(N)}]\in \mathbb{R}^{k \times N}$, where $k$ are the amount of channels and $N$ the observations. Then we take a vector $\mathbf{x}_{k}=[\mathrm{x}^{(1)}_{k},\ldots, \mathrm{x}^{(N)}_{k}]$, calculate its mean $\mu_{k}$, standard deviation $\sigma_{k}$ and normalise it as $\mathbf{\bar{x}}_{k}=[\frac{\mathrm{x}^{(1)}_{k}-\mu_{k}}{\sigma_{k}},\ldots, \frac{\mathrm{x}^{(N)}_{k}-\mu_{k}}{\sigma_{k}}]$. Repeating this for each channel, we get $\mathbf{\bar{X}}_{\mathrm{EEG}}$.

In [4]:
def _preprocessing(data):
    logger.info(f'Preprocessing data')
    normalized_data = np.empty(shape=data.shape)
    for idx, trial in enumerate(data):
        normalized_data[idx] = StandardScaler().fit_transform(X=trial)
    return normalized_data

# Feature Extraction
### One Dimension-Aggregate Approximation (1d-AX)
----
1d-AX  can calculated by taking $\mathrm{\bar{x}}_{k}=[\mathrm{\bar{x}}^{(1)}_{k},\ldots, \mathrm{\bar{x}}^{(N)}_{k}]$ for each channel, and divide it into $m$ segements $\mathbf{y}_{k}^{(i)}$ of equal length $q=\frac{N}{m}$. Afterwards, we take a sampled version of $\mathbf{y}_{k}^{(i)}$ and apply linear regression to it.

In [5]:
def _segment_data(data, seg_length):
    logger.info(f'Segmenting data')
    num_trials, num_samples, num_channels = data.shape
    assert num_samples % seg_length == 0
    num_segments = num_samples//seg_length
    seg_eeg_data = np.empty(shape=(num_trials, num_segments, seg_length, num_channels))

    for trial_idx in range(num_trials):
        for segment_idx in range(num_segments):
            lower_limit = segment_idx * seg_length
            upper_limit = lower_limit + seg_length
            seg_eeg_data[trial_idx, segment_idx] = data[trial_idx, lower_limit:upper_limit, :]

    return seg_eeg_data

In [6]:
def _linear_regression(data):
    num_trials, num_segments, num_samples, num_channels = data.shape
    x = np.linspace(1, num_samples, num_samples)

    K = np.zeros(shape=(num_trials, num_segments, num_channels))
    A = np.zeros(shape=(num_trials, num_segments, num_channels))
    for trial_idx, trial in enumerate(data):
        for segment_idx, segment in enumerate(trial):
            for channel_idx in range(num_channels):
                (c1, c0) = np.polyfit(x, segment[:, channel_idx], deg=1)
                t_mean = np.mean(segment[:, channel_idx])
                K[trial_idx, segment_idx, channel_idx] = c1
                A[trial_idx, segment_idx, channel_idx] = c1 * t_mean + c0

    return K, A

# Channel Weighting
----
#### Following the 1d-AX step, we calculate spatial filters for dimensionality reduction. The idea is to gain filters $\mathbf{W_{\mathbf{K}}}$ and $\mathbf{W_{\mathbf{A}}}$, which reduce the dimensionality of $\mathbf{K}$ and $\mathbf{A}$ by taking $\mathbf{K}'=\mathbf{W_{\mathbf{K}}}\mathbf{K}$, as well as $\mathbf{A}'=\mathbf{W_{\mathbf{A}}}\mathbf{A}$.

# LSTM and Softmax Regression
----
#### Subsequently, the output of the filters $\mathbf{K}'$, $\mathbf{A}'$ is fed into two LSTMs in parallel. Output of the LSTM is merged and fed into a Softmax Regression.

In [7]:
def _fit(data, num_reduced_channels, LSTM_cells, num_classes):
    num_samples, num_segments, num_channels = data.shape
    
    if num_classes == 4:
        Input = tf.keras.Input(shape=(num_segments, num_channels), batch_size=None)
        Input_f = tf.keras.layers.Dense(num_reduced_channels)(Input)
        Input_f_norm = tf.keras.layers.BatchNormalization()(Input_f)
        LSTM = tf.keras.layers.LSTM(LSTM_cells, dropout=0.6)(Input_f_norm)
        Output = tf.keras.layers.Dense(num_classes, activation='softmax')(LSTM)

        return tf.keras.Model(inputs=Input, outputs=Output)
    
    else:
        Input_K = tf.keras.Input(shape=(num_segments, num_channels), batch_size=None)
        Input_A = tf.keras.Input(shape=(num_segments, num_channels), batch_size=None)
        Input_K_f = tf.keras.layers.Dense(num_reduced_channels)(Input_K)
        Input_A_f = tf.keras.layers.Dense(num_reduced_channels)(Input_A)
        Input_K_f_norm = tf.keras.layers.BatchNormalization()(Input_K_f)
        Input_A_f_norm = tf.keras.layers.BatchNormalization()(Input_A_f)
        LSTM_K = tf.keras.layers.LSTM(LSTM_cells, dropout=0.6)(Input_K_f_norm)
        LSTM_A = tf.keras.layers.LSTM(LSTM_cells, dropout=0.6)(Input_A_f_norm)
        Merged_LSTM = tf.keras.layers.concatenate([LSTM_K, LSTM_A])
        Output = tf.keras.layers.Dense(num_classes, activation='softmax')(Merged_LSTM)

        return tf.keras.Model(inputs=[Input_K, Input_A], outputs=Output)

#### Training a model on a given set of training features and labels
_______

In [8]:
def _train(K, A, target, spatial_filter_dim, LSTM_cells, num_classes, epochs, learning_rate, verbose, batch_size, log_dir):
    
    model = _fit(data=K, num_reduced_channels=spatial_filter_dim, LSTM_cells=LSTM_cells, num_classes=num_classes)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    metrics = [tf.keras.metrics.CategoricalAccuracy()]
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
    callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    if num_classes == 4:
        model.fit([A], target, epochs=epochs, verbose=verbose, batch_size=batch_size, callbacks=[callback])
    
    else:
        model.fit([K, A], target, epochs=epochs, verbose=verbose, batch_size=batch_size, callbacks=[callback])
    
    return model

#### Evaluating a model on a given set of test data and labels
_______

In [9]:
def evaluate(data, target, model, segment_length, num_classes, batch_size, log_dir):
    new_target = map_to_one_hot(target)
    normalized_data = _preprocessing(data)
    segmented_data = _segment_data(normalized_data, segment_length)
    K, A = _linear_regression(segmented_data)
    callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    if num_classes == 4:
        loss, acc = model.evaluate([A], new_target, callbacks=[callback])
        pred_label = model.predict(A, batch_size=batch_size)
        
    else:
        loss, acc = model.evaluate([K, A], new_target, callbacks=[callback])
        pred_label = model.predict([K, A], batch_size=batch_size)
    
    return acc, tf.math.confusion_matrix(tf.math.argmax(new_target, axis=1),tf.math.argmax(pred_label, axis=1))

#### Pipeline for training the model on a given data and labels
______

In [10]:
def pipeline(data, target, segment_length, spatial_filter_dim, LSTM_cells, num_classes, epochs, batch_size, log_dir, verbose=0, learning_rate=3e-4):
    new_target = map_to_one_hot(target)
    normalized_data = _preprocessing(data)
    segmented_data = _segment_data(normalized_data, segment_length)
    K, A = _linear_regression(segmented_data)
    model = _train(K, A, target=new_target, 
                   spatial_filter_dim=spatial_filter_dim, LSTM_cells=LSTM_cells, 
                   num_classes=num_classes, epochs=epochs, 
                   learning_rate=learning_rate, verbose=verbose, 
                   batch_size=batch_size, log_dir=log_dir)
    return model

 #### Mapping class labels to a categorical variable.
 ______

In [11]:
def map_to_one_hot(target):
    new_target = np.copy(target)
    unique = np.unique(target)
    new_labels = np.linspace(0, len(unique)-1, len(unique))
    mapping = dict(zip(unique, new_labels))
    for idx, label in enumerate(target):
        new_target[idx] = mapping[label]
    return tf.one_hot(new_target, depth=len(unique))

# Example of a possible training scheme
-------
Define parameters if interest, i.e. LSTM cells or dimension of the spatial filter.

In [12]:
k_cross_val = 5
num_segments = 6
t_sec = 3
epochs=300
spatial_filter_dim, LSTM_cells =  6, 8
learning_rate = 3e-4
batch_size=64

for comb_idx, comb in enumerate([[769, 770], [770, 772]]):
    EEG, TARGET, _ = get_npz_data(path=BCI_IV_2a_DATA_PATH, user='A01E', labels=comb, sec=t_sec)
    for split_idx, (train_idx, test_idx) in enumerate(StratifiedKFold(n_splits=k_cross_val).split(np.zeros(shape=TARGET.shape), TARGET)):
        EEG_TRAIN, TARGET_TRAIN = EEG[train_idx], TARGET[train_idx]
        EEG_TEST, TARGET_TEST = EEG[test_idx], TARGET[test_idx]
        
        num_classes = len(set(TARGET_TRAIN))
        _, samples_per_trial, _ = EEG_TRAIN.shape
        log_dir = f'{LOG_PATH}/CIDX-{comb_idx}-SIDX-{split_idx}-{comb[0]}-{comb[1]}/'

        model = pipeline(data=EEG_TRAIN, target=TARGET_TRAIN, 
                         segment_length=samples_per_trial//num_segments, 
                         spatial_filter_dim=spatial_filter_dim, 
                         LSTM_cells=LSTM_cells, 
                         num_classes=num_classes,
                         learning_rate=learning_rate,
                         batch_size=batch_size,
                         epochs=epochs,
                         log_dir=log_dir)
        
        acc, _ = evaluate(data=EEG_TEST, target=TARGET_TEST,
                          model=model, 
                          segment_length=samples_per_trial//num_segments,
                          num_classes=num_classes,
                          batch_size=batch_size,
                          log_dir=log_dir)

Fetching npz data from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/BCI/A01E.npz

Import of BCI Competition IV 2a dataset done
Shape of data (144, 750, 22) and targets (144,)

Preprocessing data
Segmenting data
Preprocessing data
Segmenting data


1/1 [==============================] - 0s 498ms/step - loss: 0.6493 - categorical_accuracy: 0.7241


Preprocessing data
Segmenting data
Preprocessing data
Segmenting data


1/1 [==============================] - 0s 464ms/step - loss: 0.6969 - categorical_accuracy: 0.5517


Preprocessing data
Segmenting data
Preprocessing data
Segmenting data


1/1 [==============================] - 0s 459ms/step - loss: 0.7136 - categorical_accuracy: 0.4483


Preprocessing data
Segmenting data
Preprocessing data
Segmenting data


1/1 [==============================] - 0s 473ms/step - loss: 0.6984 - categorical_accuracy: 0.4483


Preprocessing data
Segmenting data


KeyboardInterrupt: 

# Hyperparameter Search
----------
#### Adapt the parameters _learning_rates_, _filter_dim_, _lstm_cells_, _num_segments_ and _epochs_ before starting the hypterparamter search.

In [59]:
GRIDSEARCH_PATH = 'LOG/Gridsearch'

learning_rates = [1e-4, 1e-3, 1e-2, 1e-1]
filter_dim = [3, 6, 9, 12]
lstm_cells = [3, 5, 8, 10]
num_segments = [3, 5, 6, 10]
epochs = [250, 500, 1000]

HP_LEARNING_RATE = hp.HParam('Learning Rate', hp.Discrete(learning_rates))
HP_FILTER_DIM = hp.HParam('Filter Dimension', hp.Discrete(filter_dim))
HP_LSTM_CELLS = hp.HParam('LSTM Cells', hp.Discrete(lstm_cells))
HP_NUM_SEGMENTS = hp.HParam('Number of Segments', hp.Discrete(num_segments))
HP_EPOCHS = hp.HParam('Epochs', hp.Discrete(epochs))
METRIC_ACCURACY = 'accuracy'


def run(path, hparams, train_data, train_labels, test_data, test_labels, batch_size, samples_per_trial, fold_idx):
        hp.hparams(hparams)
        model = pipeline(data=train_data, target=train_labels,
                         segment_length=samples_per_trial//hparams[HP_NUM_SEGMENTS], 
                         spatial_filter_dim=hparams[HP_FILTER_DIM], 
                         LSTM_cells=hparams[HP_LSTM_CELLS], 
                         num_classes=len(set(train_labels)),
                         learning_rate=hparams[HP_LEARNING_RATE],
                         batch_size=batch_size,
                         epochs=hparams[HP_EPOCHS], 
                         log_dir=path)
        
        accuracy, _ = evaluate(data=test_data, target=test_labels,
                               model=model,
                               segment_length=samples_per_trial//hparams[HP_NUM_SEGMENTS],
                               num_classes=len(set(train_labels)),
                               batch_size=batch_size,
                               log_dir=path)

        return accuracy

In [ ]:
EEG, TARGET, _ = get_npz_data(BCI_IV_2a_DATA_PATH, 'A01E')
_, samples_per_trial, _ = EEG.shape
session_idx = 0
batch_size = 64

for lr_idx, lr in enumerate(HP_LEARNING_RATE.domain.values, 0):
    for fdim_idx, fdim in enumerate(HP_FILTER_DIM.domain.values, 0):
        for cells_idx, cells in enumerate(HP_LSTM_CELLS.domain.values, 0):
            for segment_idx, N in enumerate(HP_NUM_SEGMENTS.domain.values, 0):
                for epoch_idx, epoch_N in enumerate(HP_EPOCHS.domain.values, 0):
                    hparams = {
                        HP_LEARNING_RATE: lr,
                        HP_FILTER_DIM: fdim,
                        HP_LSTM_CELLS: cells,
                        HP_NUM_SEGMENTS: N,
                        HP_EPOCHS: epoch_N
                    }
                    
                    accuracies = list()
                    path = f'{GRIDSEARCH_PATH}/grid-{session_idx}/'
                    with tf.summary.create_file_writer(path).as_default(): 
                        for fold_idx, (train_idx, test_idx) in enumerate(StratifiedKFold(n_splits=5).split(np.zeros(shape=TARGET.shape), TARGET)):
                            EEG_TRAIN, TARGET_TRAIN = EEG[train_idx], TARGET[train_idx]
                            EEG_TEST, TARGET_TEST = EEG[test_idx], TARGET[test_idx]

                            acc = run(path=f'{path}/fold-{fold_idx}', hparams=hparams, 
                                      train_data=EEG_TRAIN, train_labels=TARGET_TRAIN, 
                                      test_data=EEG_TEST, test_labels=TARGET_TEST, 
                                      batch_size=batch_size, samples_per_trial=samples_per_trial,
                                      fold_idx=fold_idx)
                            accuracies.append(acc)

                        tf.summary.scalar(METRIC_ACCURACY, tf.reduce_mean(accuracies), step=session_idx)
                        session_idx += 1

Fetching npz data from /upb/scratch/users/d/dann/1d-AX-LSTM/Data/BCI/A01E.npz

Import of BCI Competition IV 2a dataset done
Shape of data (288, 750, 22) and targets (288,)

Preprocessing data
Preprocessing data
Segmenting data
Segmenting data
Preprocessing data
Preprocessing data
Segmenting data
Segmenting data


2/2 [==============================] - 0s 2ms/step - loss: 1.3835 - categorical_accuracy: 0.2241


Preprocessing data
Preprocessing data
Segmenting data
Segmenting data
